<a href="https://colab.research.google.com/github/marzione00/Association_analysis_and_tweets/blob/main/TWEET_CSV_gen_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tweet retrieval 
Solution adapted from DSE Lab: SKY Social media observatory

import libraries

In [ ]:
import tweepy
import json
import time
import pandas as pd
import numpy as np
import datetime
import os
import re
import itertools
import requests
from google.colab import drive


Authentication for Twitter APIs

In [ ]:
config = json.loads(requests.get("https://www.dropbox.com/s/7kc5lbfxkobcmdw/lab.json?dl=1").text)

consumer_key        = config['consumer_key'] 
consumer_secret     = config['consumer_secret']
access_token_key    = config['access_token_key'] 
access_token_secret = config['access_token_secret'] 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)
api  = tweepy.API(auth)

if api.me():
    print("Authenticated\n")
else:
    print("Error in Authentication\n")

Creation of the path for the dataset

In [ ]:
drive.mount("/content/drive",force_remount=True)

base_path = "/content/drive/MyDrive/tweet_vaccines/"
path_out = "/content/drive/MyDrive/tweet_vaccines/" + str(datetime.date.today()) + '.csv'

Mounted at /content/drive


Dataset retrieval 

In [ ]:
hashtag_list_en = ["#test"] #insert the chosen hashtag
hashtag_list_it = ["#test"]

no_retweet   = " -filter:retweets"
language     = ["en"] #select the languege(s)
today        = datetime.date.today().strftime("%Y-%m-%d")
period       = " until:" + today


flag_header = True

if flag_header == True:
    with open(path_out, "w") as o:
        o.write("hashtag|timestamp|tweet_text|user_name|language\n")

    flag_header = False

counter = 0
times   = 0

while times <= 100: 

    for lang in language:

        if lang == "en":
            hashtag_list = hashtag_list_en

        else:
            hashtag_list = hashtag_list_it

        for hash in hashtag_list:

            query = hash + no_retweet + period + " lang:" + lang

            try:

                for page in tweepy.Cursor(api.search, q=query, tweet_mode = "extended").pages():
                    for status in page:

                        try:
                            full_text = status.full_text.replace("\n", " ").replace("|", "").encode("utf-8").decode("ascii", "ignore").strip().replace("  ", " ")
                            url_to_rm = re.search("(?P<url>https?://[^\s].+)", full_text).group("url") # searching for https in tweet

                            text      = full_text.replace(url_to_rm, "").strip().replace("  ", " ") # removes https in tweet
                            time_data = str(status.created_at)
                            user_name = status.user.screen_name
                            # print str(counter) + " : " + text
                            counter += 1
                            print(str(counter), ":", hash)
                        except:

                            text      = status.full_text.replace("\n", " ").replace("|", "").encode("utf-8").decode("ascii", "ignore").strip().replace("  ", " ")
                            time_data = str(status.created_at)
                            user_name = status.user.screen_name
                            # print str(counter) + " : " + text
                            counter += 1
                            print(str(counter), ":", hash)

                        raw_line = str(hash + "|" + time_data + "|" + text + "|" + user_name + "|" + lang)

                        with open(path_out, "a") as o:
                            o.write(raw_line + '\n')

                        # print(str(counter))
                        time.sleep(0.1)

            except Exception as e:
                print(e)
                print("\n\nSuperato il limite, aspetto 15 minuti")

                time.sleep(15 * 60 + 1)

    times += 1



path_out_deduplicated = path_out.replace(".csv", "_dedupl.csv")

df_tweets = pd.read_csv(path_out, sep = "|", header= 0, error_bad_lines=False)
df_tweets_no_dupl = df_tweets.drop_duplicates()
df_tweets_no_dupl.to_csv(path_out_deduplicated, index = False)

1 : #test
2 : #test
3 : #test
4 : #test
5 : #test
6 : #test
7 : #test
8 : #test
9 : #test
10 : #test
11 : #test
12 : #test
13 : #test
14 : #test
15 : #test
16 : #test
17 : #test
18 : #test
19 : #test
20 : #test
21 : #test
22 : #test
23 : #test
24 : #test
25 : #test


KeyboardInterrupt: ignored